In [1]:
# Code Author Naveen Kumar Vadlamudi
# The following program takes the schedule into consideration and then generates a feasible schedule.

import mysql.connector
from collections import defaultdict
from datetime import datetime

config = {'user':'root', 'password':'****','database':'****'}

def checkForSections(cinfo, classDayResult, timeSlot, rating):
    if len(classDayResult) > 1:
         schedule[cinfo] = [classDayResult[0], classDayResult[1], timeSlot[0], timeSlot[1],timeSlot[2].seconds, timeSlot[3].seconds, rating]
    elif cinfo in schedule:
            pass
    else:    
         schedule[cinfo] = [classDayResult[0], None, timeSlot[0], timeSlot[1], None, None, rating]

# temp is a dictionary of lists 
# classDayResult is a Single value character that tells us which date is it on.
# timeSlot is an array where we have time schedule values []
# Iterates over the existing time Schedules and will provide us an output

def iterateExistingSchedules(temp,classDayResult, timeSlot):
    for keys,val in temp.items():
            if val[1] is None:
                if(val[0] == classDayResult[0]) and (timeSlot[0] >= val[2] and timeSlot[0] <= val[3] or (timeSlot[1]>= val[2] and timeSlot[1] <= val[3])):
                                            return False
            else:
                if (                                                               
                        (val[0] == classDayResult[0]) and                          
                        (                                                          
                            (timeSlot[0] >= val[2] and timeSlot[0] <= val[3]) or   
                            (timeSlot[1]>= val[2] and timeSlot[1] <= val[3])       
                        ) or                                                     
                        (                                                          
                            val[1] == classDayResult[0] and                       
                         (  (timeSlot[0] >= val[4] and timeSlot[0] <= val[5]) or   
                            (timeSlot[1]>= val[4] and timeSlot[1] <= val[5])       
                          )                                                        
                        )                                                          
                   ):
                                            return False
    return True   
           
                
# check for the overlapping schedules by iterating the dictionary
def checkForSchedule(temp, cinfo, classDayResult, timeSlot):
    output = []
    if cinfo not in temp:
            if len(classDayResult) > 1:
                multiScheduleInput =  [
                                        [classDayResult[0],timeSlot[0],timeSlot[1]],
                                        [classDayResult[1],timeSlot[2].seconds,timeSlot[3].seconds]
                                      ]
                for ind in range(len(multiScheduleInput)):
                        output.append(iterateExistingSchedules(temp, multiScheduleInput[ind][0], [multiScheduleInput[ind][1],multiScheduleInput[ind][2]]))
                        
            else:
                        output.append(iterateExistingSchedules(temp,classDayResult,timeSlot))

    else:
            flag = False

    if False in output:        
        return False
    elif True in output:
        return True

    


if __name__ == "__main__":
    cnx = mysql.connector.connect(**config)
    cursor = cnx.cursor()
    
    query = "SELECT CINFO, CLASS_DAY, TS1_START_TIME, TS1_END_TIME, TS2_START_TIME, TS2_END_TIME, RATING \
    FROM COURSES ORDER BY RATING DESC "
    
    cursor.execute(query)
    
    schedule = defaultdict(list)
    count = 0
    for (cinfo, classday, startTime1, endTime1, startTime2, endTime2, rating) in cursor:
                    classDayResult = classday.split(',')
                    startTime1 = startTime1.seconds
                    endTime1 = endTime1.seconds
                    if len(schedule) > 0:
                            temp = schedule.copy()
                            # Checks if the schedule is present in the dictionary and returns a bool value
                            if checkForSchedule(temp,cinfo,classDayResult,[startTime1,endTime1,startTime2, endTime2]):
                                checkForSections(cinfo, classDayResult,[startTime1,endTime1,startTime2, endTime2], rating)
                    else:  
                            # initialize the dictionary with the 
                            # first or highest rating record
                            checkForSections(cinfo, classDayResult, [startTime1,endTime1,startTime2, endTime2], rating)                         
                    count += 1
                    print("iteration: ",count,end= "\n")
                    for key, value in schedule.items():
                         print(value, key, end = "\n")
                    print()
                            
    print("The Output Course Schedule While using Greedy Approach is the Following: ")
    limit = 0
    rating = 0
    for key, value in schedule.items():
            if limit < 5: 
                rating+= value[-1]
                print(value, key, end = "\n")
            limit+=1
    print()
    print("The rating of the following Schedule is :", rating)
    cursor.close()        
    cnx.close()

    

ModuleNotFoundError: No module named 'mysql'